# Data Preprocessing
### trade.csv from [관세청 무역 수출입 통계](https://unipass.customs.go.kr/ets/index.do)   

## Goal of this project
- find duplicated / missing data and remove / fill in
- normalize data
- find outliers and deal with them
- one-hot encode categorical data
- convert categorical data into continuous data by dividing it into sections
- terms: ***missing data, outlier, normalization, one-hot encoding, binning***

In [ ]:
import os

csv_file_path = os.getenv('HOME')+'/aiffel/data_preprocess/data/trade.csv'
trade = pd.read_csv(csv_file_path) 
trade.head()

# Missing Data

In [ ]:
# get the count of the missing data
len(trade) - trade.count()

# then drop the column that is empty (missing data)
trade = trade.drop('기타사항', axis=1)
trade.head()

- `DataFrame.isnull()`은 데이터마다 결측치 여부를 True, False로 반환합니다.
- `DataFrame.any(axis=1)`는 행마다 하나라도 True가 있으면 True, 그렇지 않으면 False를 반환합니다.
- `trade.isnull().any(axis=1)`을 다시 DataFrame에 넣어주면 값이 True인 데이터만 추출해줍니다.

- 우선 '수출건수', '수출금액', '수입건수', '수입금액', '무역수지' 열이 모두 결측치인 index 196, 197, 198을 삭제
- `DataFrame.dropna`는 결측치를 삭제해주는 메서드입니다.
- `subset` 옵션으로 특정 컬럼들을 선택
- `how` 옵션으로 선택한 컬럼 전부가 결측치인 행을 삭제하겠다는 의미로 'all'을 선택 ('any': 하나라도 결측치인 경우)
- `inplace` 옵션으로 해당 DataFrame 내부에 바로 적용

### 수치형 데이터의 결측값을 보완하는 방법
1. 특정 값을 지정해줄 수 있습니다. 그러나 결측치가 많은 경우, 모두 같은 값으로 대체한다면 데이터의 분산이 실제보다 작아지는 문제가 생길 수 있습니다.

2. 평균, 중앙값 등으로 대체할 수 있습니다. 1번에서 특정 값으로 대체했을 때와 마찬가지로 결측치가 많은 경우 데이터의 분산이 실제보다 작아지는 문제가 발생할 수 있습니다.

3. 다른 데이터를 이용해 예측값으로 대체할 수 있습니다. 예를 들어 머신러닝 모델로 2020년 4월 미국의 예측값을 만들고, 이 값으로 결측치를 보완할 수 있습니다.

4. 시계열 특성을 가진 데이터의 경우 앞뒤 데이터를 통해 결측치를 대체할 수 있습니다. 예를 들어 기온을 측정하는 센서 데이터에서 결측치가 발생할 경우, 전후 데이터의 평균으로 보완할 수 있습니다.

### 범주형 데이터의 결측값을 보완하는 방법
1. 특정 값을 지정해줄 수 있습니다. 예를 들어 ‘기타’, ‘결측’과 같이 새로운 범주를 만들어 결측치를 채울 수 있습니다.

2. 최빈값 등으로 대체한다. 결측치가 많은 경우 최빈값이 지나치게 많아질 수 있으므로 결측치가 많을 때는 다른 방법을 사용한다.

3. 다른 데이터를 이용해 예측값으로 대체할 수 있다.

4. 시계열 특성을 가진 데이터의 경우 앞뒤 데이터를 통해 결측치를 대체할 수 있습니다. 예를 들어 특정인의 2019년 직업이 결측치이고, 2018년과 2020년 직업이 일치한다면 그 값으로 보완할 수 있습니다. 만약 다르다면 둘 중 하나로 보완하도록 합니다.

# Duplicated data

In [ ]:
trade.duplicated() #returns boolean
trade.drop_duplicates(inplace=True) 
df.drop_duplicates(subset=['id'], keep='last') # 중복된 값 중 뒷 인덱스의 값을 지우고 싶을 경우


# Outlier
- min-max scaling
- anomaly detection
    - use `z score`: 평균과 표준편차 이용

### What to do after detecting outlier...
1. 가장 간단한 방법으로 이상치를 삭제할 수 있습니다. 이상치를 원래 데이터에서 삭제하고, 이상치끼리 따로 분석하는 방안도 있습니다.

2. 이상치를 다른 값으로 대체할 수 있습니다. 데이터가 적으면 이상치를 삭제하기보다 다른 값으로 대체하는 것이 나을 수 있습니다. 예를 들어 최댓값, 최솟값을 설정해 데이터의 범위를 제한할 수 있습니다.

3. 혹은 결측치와 마찬가지로 다른 데이터를 활용하여 예측 모델을 만들어 예측값을 활용할 수도 있습니다.


## z-score method (standard score)

In [ ]:
# z-score method
def outlier(df, col, z): # z: 기준
    return df[abs(df[col] - np.mean(df[col]))/np.std(df[col])>z].index 
    # abs(df[col] - np.mean(df[col])): 데이터에서 평균을 빼준 것의 절대값
    # /np.std(df[col]): 표준편차로 나눔
    # /z].index: 값이 z보다 큰 데이터의 인덱스 추출

def not_outlier(df, col, z):
    return df[abs(df[col] - np.mean(df[col]))/np.std(df[col]) <= z].index

trade.loc[outlier(trade, '무역수지', 1.5)
trade.loc[not_outlier(trade, '무역수지', 1.5)]


## IQR method
a compliment for z-score method (limitations..)
IQR (Interquartile range) 사분위범위수로 이상치를 찾을 수 있다
$$IQR = Q_3 - A_1$$
IQR은 제 3사분위수에서 제 1사분위 값을 뺀 값으로, 데이터의 중간 50% 범위라고 할 수 있다   
$Q_1 - 1.5 * IQR$ 보다 왼쪽에 있거나, $Q_3 _ 1.5 * IQR$보다 오른쪽에 있는 경우 이상치라고 판단

In [ ]:
Q3, Q1 = np.percentile(data, [75 ,25])
IQR = Q3 - Q1
IQR

data[(Q1-1.5*IQR > data)|(Q3+1.5*IQR < data)] # check for outliers 

article ref [Three ways to detect outliers](http://colingorrie.github.io/outlier-detection.html) is introducing `z-score` method, `modified z-score` method, and `IQR` method

# Normalization
- Standardization
$$(X-μ)/(σ)$$
- Min-Max Scaling
$$(X - X_[min])/(X_(max) - X_(min))$$

In [ ]:
# Normalize data x with standardization
x_standardization = (x - x.mean())/x.std() # set each column's mean to 0, and deviation to 1
x_standardization

# Normalize data x with min-max scaling
x_min_max = (x-x.min())/(x.max()-x.min()) # set colunn's min to 0, max to 1
x_min_max


### Standardization

In [ ]:
# Use trade data apply standardizatin method
cols = ['수출건수', '수출금액', '수입건수', '수입금액', '무역수지']
trade_Standardization= (trade[cols]-trade[cols].mean())/trade[cols].std()
trade_Standardization.head()

trade_Standardization.describe() # check for mean and the standard deviation

### Min-Max Scaling

In [ ]:
# Use trade data apply min-max scaling method
trade[cols] = (trade[cols]-trade[cols].min())/(trade[cols].max()-trade[cols].min())
trade.head()

trade.describe()

## Use scikit-learn's `StandardScaler`, `MinMaxScaler`

In [ ]:
from sklearn.preprocessing import MinMaxScaler
train = [[10, -10], [30, 10], [50, 0]]
test = [[0, 1]]
scaler = MinMaxScaler()

scaler.fit_transform(train)
scaler.transform(test)

Ref for Andrew Ng's [Normalizing inputs](https://www.youtube.com/watch?v=FDCfw-YqWTE&ab_channel=DeepLearningAI)